## Introduction

In this Project, I will be creating a Basic Level Restaurant Recommender which will give you the top Restaurants near your address. I’ll explore the venues based on their Rating and Distance. Whenever a person wants to find a Particular Restaurants in the vicinity of his current address or any address then, this model will recommend the Best Restaurants nearby on Map with their ratings. 

#### Imported necessary Libraries

In [2]:
import requests 
import pandas as pd
import numpy as np 
import random
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

import folium

In [3]:
CLIENT_ID = 'LJXD0QA23BVCO1U010AEUBTVB5F4C0KIMOVL5WX42MRCYE0K' 
CLIENT_SECRET = 'NFNLWLUMMQWIQLIZBHTF1PYMRYZGBHLUBVCQYMYOH30HSG3P'
VERSION = '20180604'
LIMIT = 20


### Enter the Address i.e. your location in the address variable to convert it to its latitude and longitude coordinates.

In [4]:
address = 'Downtown, Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude,location)

43.6563221 -79.3809161 Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada


### Enter the Query i.e. Search for a specific Restaurant category. You can either write Cafe or Restaurant if you are not sure about cuisines you can also specify the Search Radius from your location.(But do not write some large radius values)

#### Now, let's assume that you are very hungry. So, we will search for 'Restaurant' that is within 1000 metres from your given location. 

In [5]:
search_query = 'Restaurant'
radius = 1000


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LJXD0QA23BVCO1U010AEUBTVB5F4C0KIMOVL5WX42MRCYE0K&client_secret=NFNLWLUMMQWIQLIZBHTF1PYMRYZGBHLUBVCQYMYOH30HSG3P&ll=43.6563221,-79.3809161&v=20180604&query=Restaurant&radius=1000&limit=20'

#### Now we will send the GET Request to Foursquare API and fetch the results

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0ca7c90de0d9001b3ab2e7'},
 'response': {'venues': [{'id': '4ad4c05ff964a52048f720e3',
    'name': 'Hemispheres Restaurant & Bistro',
    'location': {'address': '110 Chestnut Street',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'distance': 434,
     'postalCode': 'M5G 1R3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1577887695',
    'hasPerk': False},
   {'id': '

#### Now we will filter out the relevant data which is important to us from JSON and convert it to Pandas DataFrame

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1577887695,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",434,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1577887695,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",861,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
2,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1577887695,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",1017,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN
3,4b223f5af964a520ba4424e3,Azure Restaurant & Bar,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1577887695,False,225 Front St W,43.644749,-79.385113,"[{'label': 'display', 'lat': 43.64474919591934...",1331,M5V 2X3,CA,Toronto,ON,Canada,[225 Front St W (in InterContinental Toronto C...,in InterContinental Toronto Centre,136175835,Entertainment District
4,4bd47e6fcfa7b7139f2924da,Studio Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1577887695,False,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",590,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,NaN


In [10]:

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtering the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# Storing column names by keeping only last term and not location
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",434,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",861,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",1017,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3
3,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,"[{'label': 'display', 'lat': 43.64474919591934...",1331,M5V 2X3,CA,Toronto,ON,Canada,[225 Front St W (in InterContinental Toronto C...,in InterContinental Toronto Centre,Entertainment District,4b223f5af964a520ba4424e3
4,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",590,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,4bd47e6fcfa7b7139f2924da
5,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",1040,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
6,Cottage Restaurant & Lounge,Thai Restaurant,338 Jarvis St.,43.662770,-79.376894,"[{'label': 'display', 'lat': 43.66277006654972...",787,M4Y 2G6,CA,Toronto,ON,Canada,"[338 Jarvis St. (north of Carlton Street), Tor...",north of Carlton Street,NaN,4ad4c05ff964a52018f720e3
7,Akashiro Japanese Restaurant & Bar,Sushi Restaurant,220 Yonge St.,43.655965,-79.380541,"[{'label': 'display', 'lat': 43.65596504293232...",49,NaN,CA,Toronto,ON,Canada,[220 Yonge St. (in Toronto Eaton Centre Urban ...,in Toronto Eaton Centre Urban Eatery,NaN,4fbbb3ace4b01e39077b4d55
8,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",1303,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,NaN,4aef8854f964a5201cd921e3
9,The Senator Restaurant,Diner,249 Victoria Street,43.655641,-79.379199,"[{'label': 'display', 'lat': 43.65564091455335...",157,M5B 1T8,CA,Toronto,ON,Canada,[249 Victoria Street (btwn Dundas St E and Shu...,btwn Dundas St E and Shuter St,NaN,4ad7929cf964a520500c21e3


#### Let's visualize the Searched Venues that are nearby by plotting them to Map

In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) 

# added a red circle marker that depicts current location
folium.features.RegularPolygonMarker(
    [latitude, longitude],
    radius=12,
    color='black',
    popup='Your Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 1
).add_to(venues_map)

# added the restaurants to map
for lat, lng, label1,label2,label3 in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories,dataframe_filtered.distance,dataframe_filtered.name):
    folium.Marker(
        [lat, lng],
        popup=label3+"\n"+label1+"\n"+str((float(label2)/1000))+' KM',
    ).add_to(venues_map)

venues_map

<a id="item2"></a>

## Now we will Explore All the nearby venues


In [12]:
venue_id = list(dataframe_filtered['id'])
venue_id


['4ad4c05ff964a52048f720e3',
 '4ad4c05cf964a52006f620e3',
 '4ada5d5bf964a520e92121e3',
 '4b223f5af964a520ba4424e3',
 '4bd47e6fcfa7b7139f2924da',
 '5750b013498e755287c6de97',
 '4ad4c05ff964a52018f720e3',
 '4fbbb3ace4b01e39077b4d55',
 '4aef8854f964a5201cd921e3',
 '4ad7929cf964a520500c21e3',
 '4b295e10f964a520ba9d24e3',
 '4d06c1ccc2e53704428cc567',
 '4edbccaaf9abeea3db5268df',
 '4ad4c05df964a5203ff620e3',
 '5a8b24b2018cbb6c4ca70dae',
 '4ad4c060f964a52070f720e3',
 '53ae6660498e1ac3c1282743',
 '5a7c3e079de23b294a2c4117',
 '52a7ae41498eed3af4d0a3fa',
 '539c6f13498e06f4cc765165']

### We will Fetch The Rating of every Restaurant nearby and store it in rating

In [13]:
rating=[]

for venue in venue_id:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    print(result)
    try:
        rating.append(result['response']['venue']['rating'])
    except:
        rating.append(0.0)

rating

{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca854bae9a21c9d062352'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca865006dce001b3a0d5d'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca8b51e152c001bd143c8'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca8c6216785001b3f6576'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca8c60de0d9001b3f5d42'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca8a6b9a389001bc1263e'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e0ca878e826ac001eef6c58'}, '

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### Now we will add the rating column to Dataframe and Sort the data frame according to rating

In [14]:
dataframe_filtered['rating']=rating
dataframe_filtered.head()


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id,rating
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",434,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3,0.0
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",861,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3,0.0
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",1017,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3,0.0
3,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,"[{'label': 'display', 'lat': 43.64474919591934...",1331,M5V 2X3,CA,Toronto,ON,Canada,[225 Front St W (in InterContinental Toronto C...,in InterContinental Toronto Centre,Entertainment District,4b223f5af964a520ba4424e3,0.0
4,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",590,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,4bd47e6fcfa7b7139f2924da,0.0


In [15]:
dataframe_filtered.sort_values(['rating'],ascending=False,inplace=True)
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id,rating
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",434,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3,0.0
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",861,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3,0.0
18,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,43.655281,-79.385337,"[{'label': 'display', 'lat': 43.65528126342919...",374,NaN,CA,Toronto,ON,Canada,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...",Dundas St.,NaN,52a7ae41498eed3af4d0a3fa,0.0
17,Restaurant | Adelaide,Restaurant,325 Bay St (31st floor),43.649563,-79.380590,"[{'label': 'display', 'lat': 43.649563, 'lng':...",752,M5H,CA,Toronto,ON,Canada,"[325 Bay St (31st floor), Toronto ON M5H, Canada]",NaN,NaN,5a7c3e079de23b294a2c4117,0.0
16,Castries Bar & Five Way Restaurant,Caribbean Restaurant,NaN,43.654728,-79.380928,"[{'label': 'display', 'lat': 43.65472793579101...",177,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,53ae6660498e1ac3c1282743,0.0


### Now after all the scraping of Data, We will have our final Dataframe as dataframe_final

In [16]:
# Will recommend Top 6 places on Map
dataframe_filtered.reset_index(drop=True,inplace=True)
dataframe_final=dataframe_filtered.loc[:5,['name','categories','address','lat','lng','distance','rating']]
dataframe_final

,name,categories,address,lat,lng,distance,rating
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,434,0.0
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,861,0.0
2,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,43.655281,-79.385337,374,0.0
3,Restaurant | Adelaide,Restaurant,325 Bay St (31st floor),43.649563,-79.380590,752,0.0
4,Castries Bar & Five Way Restaurant,Caribbean Restaurant,NaN,43.654728,-79.380928,177,0.0
5,Adega Restaurant,Restaurant,33 Elm St,43.657519,-79.383462,244,0.0


In [17]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent your location
folium.features.RegularPolygonMarker(
    [latitude, longitude],
    radius=10,
    color='black',
    popup='Your Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 1
).add_to(venues_map)

# add the restaurants as blue markers
for lat, lng, label1,label2,label3,label4 in zip(dataframe_final.lat, dataframe_final.lng, dataframe_final.categories,dataframe_final.distance,dataframe_final.name,dataframe_final.rating):
    if label4==0.0:
        label4='Unrated'
    folium.Marker(
        [lat, lng],
        popup=label3+"\n"+label1+"\n"+"Rating: "+str(label4),
    ).add_to(venues_map)

# display map
venues_map

### Thus Red Rectangle is your Current Location and BlueMarkers are Top 6 Restaurants based on your search Query with Name, Category and Rating of the Restaurant.